In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from tensorflow. keras.models import Sequential

# 체크포인트를 만드는 라이브러리 부분 임포트
from tensorflow.keras.callbacks import ModelCheckpoint

import os
import cv2

In [2]:
# 이미지 사이즈 조절하여 저장 하는 함수

def img_resize_save(img_path, save_path, image_shape) :
    img_list = os.listdir(img_path)
    count = 0
    for img in img_list :
        image = cv2.imread(img_path + img)
        
        image = cv2.resize(image, image_shape)
        
        cv2.imwrite(save_path+img, image)
        count += 1

In [3]:
img_resize_save('./data/cut_image/','./data/cut_image_244/',(244, 244))

In [4]:
df = pd.read_csv('./data/label.csv')
df

,file_name,label
0,cut_helmet_0.png,With Helmet
1,cut_helmet_1.png,With Helmet
2,cut_helmet_10.png,With Helmet
3,cut_helmet_100.png,Without Helmet
4,cut_helmet_1000.png,Without Helmet
...,...,...
2064,cut_helmet_995.png,Without Helmet
2065,cut_helmet_996.png,Without Helmet
2066,cut_helmet_997.png,Without Helmet
2067,cut_helmet_998.png,With Helmet


In [5]:
image_list = os.listdir('./data/cut_image_244/')
image_list[:10]

['cut_helmet_0.png',
 'cut_helmet_1.png',
 'cut_helmet_10.png',
 'cut_helmet_100.png',
 'cut_helmet_1000.png',
 'cut_helmet_1001.png',
 'cut_helmet_1002.png',
 'cut_helmet_1003.png',
 'cut_helmet_1004.png',
 'cut_helmet_1005.png']

In [6]:
image_data = []
for img in image_list :
    image = cv2.imread('./data/cut_image_244/' + img)
    image_data.append(image)

In [7]:
len(image_data)

2069

In [8]:
image_data = np.array(image_data)
image_data.shape

(2069, 244, 244, 3)

In [9]:
image_label = df['label']
image_label[:10]

0       With Helmet
1       With Helmet
2       With Helmet
3    Without Helmet
4    Without Helmet
5    Without Helmet
6       With Helmet
7       With Helmet
8       With Helmet
9       With Helmet
Name: label, dtype: object

In [10]:
y = [1 if i == 'With Helmet' else 0 for i in image_label]
y[:10]

[1, 1, 1, 0, 0, 0, 1, 1, 1, 1]

In [11]:
x = image_data.copy()

In [12]:
x.shape

(2069, 244, 244, 3)

In [13]:
x = x.astype('float32') / 255
x[0]

array([[[0.9647059 , 0.9647059 , 0.9647059 ],
        [0.9647059 , 0.9647059 , 0.9647059 ],
        [0.9647059 , 0.9647059 , 0.9647059 ],
        ...,
        [0.84705883, 0.85490197, 0.8509804 ],
        [0.8862745 , 0.88235295, 0.8784314 ],
        [0.9019608 , 0.8980392 , 0.89411765]],

       [[0.9647059 , 0.9647059 , 0.9647059 ],
        [0.9647059 , 0.9647059 , 0.9647059 ],
        [0.9647059 , 0.9647059 , 0.9647059 ],
        ...,
        [0.8352941 , 0.84313726, 0.8352941 ],
        [0.87058824, 0.87058824, 0.8627451 ],
        [0.8862745 , 0.88235295, 0.8784314 ]],

       [[0.9647059 , 0.9647059 , 0.9647059 ],
        [0.9647059 , 0.9647059 , 0.9647059 ],
        [0.9647059 , 0.9647059 , 0.9647059 ],
        ...,
        [0.8156863 , 0.827451  , 0.81960785],
        [0.85490197, 0.8509804 , 0.84705883],
        [0.87058824, 0.8666667 , 0.85882354]],

       ...,

       [[0.23529412, 0.2627451 , 0.2509804 ],
        [0.23137255, 0.25882354, 0.24705882],
        [0.22745098, 0

In [14]:
y = np.array(y)

In [15]:
### 데이터 증강



In [16]:
model = Sequential(name = 'VGG16_custom')

model.add(Conv2D(input_shape=(244,244,3), filters = 64, kernel_size = (3,3), activation = 'relu', padding='same'))

# 1층
model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same', name='1st_conv1')) # padding 데이터를 압축하는 옵션
model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same', name='1st_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2))) # strides 데이터를 압축하는 옵션

# 2층
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', name='2nd_conv1'))
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', name='2nd_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# 3층
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', name='3rd_conv1'))
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', name='3rd_conv2'))
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', name='3rd_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# 4층
model.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same', name='4th_conv1'))
model.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same', name='4th_conv2'))
model.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same', name='4th_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# 5층
model.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same', name='5th_conv1'))
model.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same', name='5th_conv2'))
model.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same', name='5th_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dense(units = 1000, activation = 'relu'))

model.add(Dense(units = 1, activation = 'sigmoid'))

In [17]:
model.summary()

Model: "VGG16_custom"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 244, 244, 64)      1792      
                                                                 
 1st_conv1 (Conv2D)          (None, 244, 244, 64)      36928     
                                                                 
 1st_conv2 (Conv2D)          (None, 244, 244, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 122, 122, 64)     0         
 )                                                               
                                                                 
 2nd_conv1 (Conv2D)          (None, 122, 122, 128)     73856     
                                                                 
 2nd_conv2 (Conv2D)          (None, 122, 122, 128)     147584    
                                                      

In [18]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x, y, validation_split=0.25, epochs = 1000, batch_size = 20)

Epoch 1/1000
78/78 [==============================] - 42s 352ms/step - loss: 1.1859 - accuracy: 0.5255 - val_loss: 0.6803 - val_accuracy: 0.6293
Epoch 2/1000
78/78 [==============================] - 20s 261ms/step - loss: 0.6932 - accuracy: 0.5177 - val_loss: 0.6899 - val_accuracy: 0.6293
Epoch 3/1000
78/78 [==============================] - 20s 261ms/step - loss: 0.6930 - accuracy: 0.5177 - val_loss: 0.6879 - val_accuracy: 0.6293
Epoch 4/1000
78/78 [==============================] - 20s 260ms/step - loss: 0.6929 - accuracy: 0.5177 - val_loss: 0.6880 - val_accuracy: 0.6293
Epoch 5/1000
78/78 [==============================] - 20s 261ms/step - loss: 0.6928 - accuracy: 0.5177 - val_loss: 0.6879 - val_accuracy: 0.6293
Epoch 6/1000
78/78 [==============================] - 20s 261ms/step - loss: 0.6928 - accuracy: 0.5177 - val_loss: 0.6865 - val_accuracy: 0.6293
Epoch 7/1000
16/78 [=====>........................] - ETA: 14s - loss: 0.6946 - accuracy: 0.4781

KeyboardInterrupt: 